In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

### Importing and Labeling dataset

In [4]:
dataset = r'D:/Downloads/Emotion detection dataset'

In [5]:
def create_label(base_dir):
    data = []
    emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

    for emotion in emotions:
        emotion_dir = os.path.join(base_dir, emotion)
        
        # Iterate through all images in the emotion folder
        for img in os.listdir(emotion_dir):
            if img.endswith('.jpg'):  # Add supported image extensions
                img_path = os.path.join(emotion_dir, img)
                data.append([img_path, emotion])

    # Create a DataFrame with two columns: 'image_path' and 'emotion'
    df = pd.DataFrame(data, columns=['image_path', 'emotion'])
    return df

In [6]:
train_set = create_label(os.path.join(dataset, 'train'))
test_set = create_label(os.path.join(dataset, 'test'))

In [7]:
train_set.to_csv('train_labels',index = False)

In [8]:
test_set.to_csv('test_labels',index = False)

In [9]:
train_set

,image_path,emotion
0,D:/Downloads/Emotion detection dataset\train\a...,angry
1,D:/Downloads/Emotion detection dataset\train\a...,angry
2,D:/Downloads/Emotion detection dataset\train\a...,angry
3,D:/Downloads/Emotion detection dataset\train\a...,angry
4,D:/Downloads/Emotion detection dataset\train\a...,angry
...,...,...
28704,D:/Downloads/Emotion detection dataset\train\s...,surprise
28705,D:/Downloads/Emotion detection dataset\train\s...,surprise
28706,D:/Downloads/Emotion detection dataset\train\s...,surprise
28707,D:/Downloads/Emotion detection dataset\train\s...,surprise


In [10]:
train_df = pd.read_csv('train_labels')
test_df = pd.read_csv('test_labels')

In [11]:
train_df

,image_path,emotion
0,D:/Downloads/Emotion detection dataset\train\a...,angry
1,D:/Downloads/Emotion detection dataset\train\a...,angry
2,D:/Downloads/Emotion detection dataset\train\a...,angry
3,D:/Downloads/Emotion detection dataset\train\a...,angry
4,D:/Downloads/Emotion detection dataset\train\a...,angry
...,...,...
28704,D:/Downloads/Emotion detection dataset\train\s...,surprise
28705,D:/Downloads/Emotion detection dataset\train\s...,surprise
28706,D:/Downloads/Emotion detection dataset\train\s...,surprise
28707,D:/Downloads/Emotion detection dataset\train\s...,surprise


In [12]:
test_df

,image_path,emotion
0,D:/Downloads/Emotion detection dataset\test\an...,angry
1,D:/Downloads/Emotion detection dataset\test\an...,angry
2,D:/Downloads/Emotion detection dataset\test\an...,angry
3,D:/Downloads/Emotion detection dataset\test\an...,angry
4,D:/Downloads/Emotion detection dataset\test\an...,angry
...,...,...
7173,D:/Downloads/Emotion detection dataset\test\su...,surprise
7174,D:/Downloads/Emotion detection dataset\test\su...,surprise
7175,D:/Downloads/Emotion detection dataset\test\su...,surprise
7176,D:/Downloads/Emotion detection dataset\test\su...,surprise


### Preprocessing the data

In [14]:
!pip install opencv-python


In [15]:
!pip install tensorflow

In [16]:
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Parameters
image_size = (48, 48)  
num_emo = 7  # Number of emotion categories

# Function to preprocess grayscale images
def preprocess_images(df):
    images = []
    labels = []
    
    for index, row in df.iterrows():
        # Load image (already in grayscale)
        img = cv2.imread(row['image_path'], cv2.IMREAD_GRAYSCALE)
        
        # Resize image to the target size
        img_resized = cv2.resize(img, image_size)
        
        # Normalize pixel values (0 to 255 -> 0 to 1)
        img_normalized = img_resized / 255.0
        
        # Append image and its corresponding label
        images.append(img_normalized)
        labels.append(row['emotion'])

    # Convert images and labels to numpy arrays
    images = np.array(images).reshape(-1, image_size[0], image_size[1], 1)  # 1 -> grayscale
    labels = np.array(labels)

    return images, labels

# Load train and test datasets
X_train, y_train = preprocess_images(train_df)
X_test, y_test = preprocess_images(test_df)

# One-hot encode the labels
y_train_encoded = to_categorical(pd.factorize(y_train)[0], num_classes=num_emo)
y_test_encoded = to_categorical(pd.factorize(y_test)[0], num_classes=num_emo)

# Split training data into train and test(validation) sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_encoded, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Validation(X_val) data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")


Training data shape: (22967, 48, 48, 1)
Validation(X_val) data shape: (5742, 48, 48, 1)
Test data shape: (7178, 48, 48, 1)


## Model Training

#### 1. Convolutional Neural Networks (CNNs)
CNN Architecture: Traditional CNN architectures (like VGG16, ResNet, or Inception) can be fine-tuned for emotion detection tasks. These models are effective for feature extraction from images.
Transfer Learning: Pre-trained models on large datasets (like ImageNet) can be fine-tuned on your specific emotion dataset for better performance.
#### 2. Facial Landmark Detection + CNN
Combine facial landmark detection (e.g., using Dlib or OpenCV) to extract key points on the face, then use those landmarks as input features to a CNN. This can improve performance by focusing on relevant facial regions.
#### 3. Recurrent Neural Networks (RNNs)
LSTM/GRU Models: For Video Data , using RNNs with Long Short-Term Memory (LSTM) or Gated Recurrent Unit (GRU) can capture temporal dependencies in facial expressions.
#### 4. Hybrid Models
Combining CNNs with RNNs (e.g., CNN followed by LSTM) can leverage spatial and temporal features, particularly useful in video emotion detection.
#### 5. Ensemble Methods
Using ensemble techniques that combine multiple models can improve accuracy and robustness in emotion detection tasks.

## Evaluation Metrics
When evaluating the performance of your model, consider using metrics like:

1. Accuracy
2. Precision, Recall, and F1 Score
3. Confusion Matrix
4. AUC-ROC Curve

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential()

# First convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output from the convolutional layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer (one node for each emotion category)
model.add(Dense(num_emo, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


C:\Users\Asus\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 46, 46, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 21, 21, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 355,847 (1.36 MB)

 Trainable params: 355,847 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

### Model Training

In [21]:
det = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=30, 
                    batch_size=32)

# Save the trained model for future use
model.save('emotion_detection_model.h5')


Epoch 1/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.2551 - loss: 1.8037 - val_accuracy: 0.3542 - val_loss: 1.5833
Epoch 2/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.3948 - loss: 1.5588 - val_accuracy: 0.4586 - val_loss: 1.4102
Epoch 3/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.4638 - loss: 1.4014 - val_accuracy: 0.5000 - val_loss: 1.3402
Epoch 4/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.4923 - loss: 1.3243 - val_accuracy: 0.5171 - val_loss: 1.2708
Epoch 5/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.5247 - loss: 1.2525 - val_accuracy: 0.5282 - val_loss: 1.2433
Epoch 6/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.5418 - loss: 1.2092 - val_accuracy: 0.5392 - val_loss: 1.2197
Epoch 7/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.5636 - loss: 1.1562 - val_accuracy: 0.5418 - val_loss: 1.2114
Epoch 8/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.5808 - loss: 1.1112 - 

In [22]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5103 - loss: 2.0811
Test accuracy: 55.60%


### Make Predictions

In [47]:
def predict_emotion(image_path):
 
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, image_size)
    img_normalized = img_resized / 255.0
    img_reshaped = img_normalized.reshape(1, 48, 48, 1)

    prediction = model.predict(img_reshaped)
    emotion = np.argmax(prediction)
    
    emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    return emotions[emotion]

# Example prediction
image_path = r"D:/Downloads/Emotion detection dataset/test/surprise/PublicTest_99446963.jpg"
predicted_emotion = predict_emotion(image_path)
print(f"Predicted emotion: {predicted_emotion}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted emotion: surprise
